import statements and data you need

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline


In [10]:
# dataframe for first csv

# Path to the data - select the path that works for you 
file_path = '../data-mining-semester-project/data/Census race.csv'

# Load the data into a DataFrame 
dfCensus = pd.read_csv(file_path)

dfCensus = dfCensus.dropna()
dfCensus= dfCensus.replace('!!',' ', regex=True)
dfCensus.head()

,GEO_ID,NAME,P1_001N,P1_002N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N,...,P1_062N,P1_063N,P1_064N,P1_065N,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N
0,id,Geographic Area Name,Total:,Total: Population of one race:,Total: Population of one race: White alone,Total: Population of one race: Black or Afri...,Total: Population of one race: American Indi...,Total: Population of one race: Asian alone,Total: Population of one race: Native Hawaii...,Total: Population of one race: Some Other Ra...,...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...,Total: Population of two or more races: Popu...
1,0500000US01001,"Autauga County, Alabama",58805,55648,42160,11445,217,881,35,910,...,0,2,2,0,0,0,0,0,0,0
2,0500000US01003,"Baldwin County, Alabama",231767,216743,189399,18217,1582,2067,143,5335,...,0,0,0,0,0,0,0,0,0,0
3,0500000US01005,"Barbour County, Alabama",25223,24523,11317,11933,116,117,1,1039,...,0,2,2,0,0,0,0,0,0,0
4,0500000US01007,"Bibb County, Alabama",22293,21534,16555,4413,60,32,9,465,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# dataframe for second csv file 
file_path2 = '../data-mining-semester-project/data/Unemployment.csv'

# Load the data into a DataFrame 
dfUnemployment = pd.read_csv(file_path2)
dfUnemployment = dfUnemployment.dropna()

dfUnemployment.head()

,FIPS_Code,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,"21,861","20,971",890,4.1,...,"26,281","25,561",720,2.7,"25,838","24,576","1,262",4.9,"58,233",112.5
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,"69,979","67,370","2,609",3.7,...,"97,424","94,713","2,711",2.8,"96,763","91,338","5,425",5.6,"59,871",115.6
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,"11,449","10,812",637,5.6,...,"8,502","8,177",325,3.8,"8,587","7,982",605,7.0,"35,972",69.5
5,1007,AL,"Bibb County, AL",1.0,1.0,1.0,"8,623","8,160",463,5.4,...,"8,639","8,371",268,3.1,"8,640","8,067",573,6.6,"47,918",92.6
6,1009,AL,"Blount County, AL",1.0,1.0,1.0,"25,266","24,375",891,3.5,...,"25,196","24,516",680,2.7,"24,661","23,653","1,008",4.1,"52,902",102.2


In [22]:
# VSRR_Provisional_Drug_Overdose_Death_Counts.csv
file_path3 = '../data-mining-semester-project/data/VSRR_Provisional_Drug_Overdose_Death_Counts.csv'

# Load the data into a DataFrame 
dfODeath = pd.read_csv(file_path3)
dfODeath = dfODeath.dropna()

dfODeath.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 18914 entries, 10 to 41624
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   State                          18914 non-null  object 
 1   Year                           18914 non-null  int64  
 2   Month                          18914 non-null  object 
 3   Period                         18914 non-null  object 
 4   Indicator                      18914 non-null  object 
 5   Data Value                     18914 non-null  object 
 6   Percent Complete               18914 non-null  object 
 7   Percent Pending Investigation  18914 non-null  float64
 8   State Name                     18914 non-null  object 
 9   Footnote                       18914 non-null  object 
 10  Footnote Symbol                18914 non-null  object 
 11  Predicted Value                18914 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory us

In [24]:
# dataframe for fourth csv file 

# Underlying Cause of Death, 1999-2019.csv
file_path4 = '../data-mining-semester-project/data/Underlying Cause of Death, 1999-2019.csv'

# Load the data into a DataFrame 
dfCDeath = pd.read_csv(file_path4)
dfCDeath.head()
dfCDeath.info()


,County,County Code,Drug/Alcohol Induced,Drug/Alcohol Induced Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Crude Rate Standard Error,Age Adjusted Rate,Age Adjusted Rate Lower 95% Confidence Interval,Age Adjusted Rate Upper 95% Confidence Interval,% of Total Deaths
0,"Autauga County, AL",1001,All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,536.0,55869.0,959.4,878.2,1040.6,41.4,812.2,742.7,881.7,0.0%
1,"Baldwin County, AL",1003,Drug-induced causes,D,Drug poisonings (overdose) Unintentional (X40-...,D1,26.0,223234.0,11.6,7.6,17.1,2.3,13,8.3,19.3,0.0%
2,"Baldwin County, AL",1003,Alcohol-induced causes,A,All other alcohol-induced causes,A9,27.0,223234.0,12.1,8.0,17.6,2.3,10.8,6.9,16.0,0.0%
3,"Baldwin County, AL",1003,All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,2222.0,223234.0,995.4,954.0,1036.8,21.1,721.3,690.2,752.3,0.1%
4,"Barbour County, AL",1005,All other non-drug and non-alcohol causes,O,All other non-drug and non-alcohol causes,O9,330.0,24686.0,1336.8,1192.6,1481.0,73.6,1021.4,907.8,1135.0,0.0%
